# Adversarial Patch - Adversarial Attacks
### Paper link : https://arxiv.org/abs/1712.09665

## Objectives & Context

Unlike classical attacks like **FGSM**, **PGD**, or **Carlini & Wagner**, which create **imperceptible perturbations** across the entire image, the **Adversarial Patch** attack is different.

It generates a **small visible square** ("patch") that can be **pasted anywhere on an image** to fool the model.

> **Goal**: Learn a single, universal patch that, when applied to any image, **forces the model to predict a specific target class**, regardless of the original content.

This attack **does not train the model**, but rather **trains the patch itself**.

---

We optimize a patch so that it **takes control over the model’s predictions**, without modifying the rest of the image.

---

### What makes *Adversarial Patch* unique?

| Property                 | Description                                                                 |
|--------------------------|-----------------------------------------------------------------------------|
| **Visible**            | The patch is **intentionally visible** (not hidden like FGSM).             |
| **Position-agnostic**  | It can be pasted **anywhere** on the image.                                |
| **Universal**          | Works on **multiple images**, not one patch per image.                     |
| **Targeted**           | Forces the prediction towards a **specific class** (e.g., *guacamole*).    |
| **Physically plausible** | Can be **printed and used in the real world** (like a sticker!).           |

---

### What does the Feature Adversaries attack do?

Here's what the attack does step-by-step:

1. Take a **source image** $I_s$ (e.g., a cat),
2. Take a **guide image** $I_g$ (e.g., a dog),
3. Generate a **modified image** $I_\alpha$, visually similar to $I_s$,
4. Make $\phi_k(I_\alpha) \approx \phi_k(I_g)$, where $\phi_k$ is the feature extractor at layer $k$.

> The network will "believe" that $I_\alpha$ is closer to $I_g$ than to $I_s$—**internally**, not visually.

---

### Intuitive Idea

Imagine pasting a **magic sticker** on any photo, and each time a model sees it, it says:

> "**This is guacamole!**" — even if it’s a lion, a car, or a cactus.

That’s exactly what **Adversarial Patch** does.

---

### How is the patch trained?

**Using backpropagation**.

The patch is treated as a **learnable parameter**, just like training weights in a network.  
We apply **gradient descent directly on the patch**.

### Steps:

1. **Initialize** the patch $p$ randomly (as pixel values),
2. **Apply** the patch to a batch of training images,
3. **Feed** the patched images into the model,
4. **Compute** the loss between model predictions and the **target class** $y_{\text{target}}$,
5. **Backpropagate** the loss and update the patch $p$.

---

### Mathematical Formula

We optimize the following loss:

$$
\min_p \; \mathbb{E}_{x \sim D} \left[ \mathcal{L} \left( f(\text{ApplyPatch}(x, p)), y_{\text{target}} \right) \right]
$$

Where:

- $p$ is the **patch** (to optimize),
- $x$ is an image sampled from the dataset $D$,
- $\text{ApplyPatch}(x, p)$ is the function that applies the patch $p$ to image $x$ (with transformations),
- $f$ is the **model** (e.g., ResNet50),
- $\mathcal{L}$ is the **loss function** (e.g., softmax cross-entropy),
- $y_{\text{target}}$ is the **desired target class** (e.g., "guacamole").

---

The result is a **universal targeted perturbation**, designed to fool the model **regardless of the background image**.

---

### Important Concept: EoT (Expectation Over Transformation)

During training, we apply **random transformations** to the patch before applying it:

- **Rotation** (e.g., from -22.5° to +22.5°),
- **Scale** (e.g., 40% to 100% of image size),
- **Position** (random location within the image).

**Why?**  
To make the patch **robust in the real world**, even if it appears at different positions or angles.

This technique is called **EoT** — *Expectation Over Transformation*:  
> "Train over many transformed versions of the patch to make it invariant."

---

### Patch Application Function

There’s no complex optimization at inference time — we just **apply the patch** to the image.

### Formula:

$$
x_{\text{patched}} = \text{ApplyPatch}(x, p, \text{position}, \text{scale}, \text{rotation})
$$

Where:

- $x$ is a **clean image**,
- $p$ is the **trained patch**,
- Parameters define **where** and **how** to apply the patch:
  - Examples: *bottom right corner*, *scale = 0.5*, *rotation = 10°*, etc.

---

# Code
### **AUTHOR** : Maxence QUINET (University Of Luxembourg)

## 1. Setup & Configuration

------------------

Please ensure all dependencies are installed using the `requirements.txt` file.

For additional environment setup details, refers to **"environment_configuration.txt"**.

-------------------

Below are the required **libraries and frameworks** for running Adversarial Attacks

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # (Reduce TensorFlow logs)
import tensorflow as tf
import torch
import numpy as np
import matplotlib.pyplot as plt

--------------------

**Machine Learning & Neural Network Libraries**

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input, Lambda, BatchNormalization, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import SGD

------------------------
**Datasets & Image Processing**

In [ ]:
from tensorflow.keras.datasets import mnist, cifar10, cifar100
from tensorflow.keras.preprocessing.image import load_img, img_to_array

-----------------
**Adversarial Robustness Toolbox (ART)**

In [ ]:
from art.estimators.classification import TensorFlowV2Classifier, PyTorchClassifier
from art.attacks.evasion import AdversarialPatch

------------------------------
**Vision Models**

In [ ]:
from PIL import Image

------------------------

**imagenet_stubs** 

imagenet_stubs is a small dataset available at this link : https://github.com/nottombrown/imagenet-stubs

#### Why use it ?

* Ideal for **testing adversarial attacks quickly** before applying them on larger datasets.
* Provides **two useful functions**:
  - `label_to_name(index)` --> Convert an ImageNet label (number) to its corresponding name
  - `name_to_label(name)` --> Convert an ImageNet class name back to its numerical label 

In [ ]:
import imagenet_stubs
from imagenet_stubs.imagenet_2012_labels import label_to_name, name_to_label

## Checking PyTorch & TensorFlow Environment

### **CUDA & GPU Verification**
Since we need **CUDA** for accelerated deep learning computations, we ensure that **PyTorch and TensorFlow** are properly configured with CUDA.

------------------
**PyTorch**

In [ ]:
 # Versions
print("PyTorch version:", torch.__version__)
print("TensorFlow version:", tf.__version__)

In [ ]:
# For PyTorch
print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: ', device)

------------------
**TensorFlow**

In [ ]:
# For Tensorflow
print(tf.config.list_physical_devices('GPU'))
tf.test.is_gpu_available()

In [ ]:
# Check CUDA & CUDNN Version
print("CUDA available:", tf.test.is_built_with_cuda())
print(tf.sysconfig.get_build_info()["cuda_version"])
print(tf.sysconfig.get_build_info()["cudnn_version"])

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(e)


## Dataset Selection & Configuration

In this section, you can **choose the dataset** you want to use for the Adversarial Patch attack:

- **MNIST**
- **CIFAR10**
- **CIFAR100**
- **ImageNet** **<--**

> **IMPORTANT NOTE**: The **official Adversarial Patch paper** explicitly tested the method on **ImageNet** only.  
> They focused on **large-scale, high-resolution images**, since the patch needs to occupy a *salient* region in the image to be effective.

> **Low-resolution datasets like MNIST or CIFAR10 may not produce reliable results** with this attack,  
> because the patch needs space to apply rotation, scaling, and random placement.

> Recommended: Use **ImageNet** with pre-trained **ResNet50**, **VGG16**, or **InceptionV3**.

---

### **Select Your Dataset & Model Configuration**

Update the variables below to set the **dataset** and the **pre-trained model** you want to use.
We recommend starting with:

```python
selected_dataset = "ImageNet"
selected_attack = "AdversarialPatch"

In [ ]:
selected_dataset = "ImageNet" # OPTIONS : "MNIST", "CIFAR10", "CIFAR100", and "ImageNet"

selected_attack = "AdversarialPatch" # Used for report name only.

--------------------------------------------
**Define class labels for each dataset**

In [ ]:
# Creation of the ancestors_name & ancestors_label corresponding to the selected dataset.

# Note: All labels are available on Internet. They are not created from us. They are official, often in a .json format.
if selected_dataset == "CIFAR10":
    # Correspondance between name & label for CIFAR10
    ancestors_name = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    ancestors_label = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

elif selected_dataset == "CIFAR100":
    # Correspondance between name & label for CIFAR100
    ancestors_name = ['apple', 'bridge', 'castle', 'elephant', 'house', 'orange', 'shark', 'table', 'tractor', 'whale']
    ancestors_label = ['0', '12', '17', '31', '37', '53', '73', '84', '89', '95']

elif selected_dataset == "ImageNet":
    # Correspondance between name & label for ImageNet
    ancestors_name = ['abacus', 'acorn', 'baseball', 'broom', 'brown_bear', 'canoe', 'hippopotamus', 'llama', 'maraca', 'mountain_bike']
    ancestors_label = ['398', '988', '429', '462', '294', '472', '344', '355', '641', '671']

elif selected_dataset == "MNIST":
    # Correspondance between name & label for ImageNet
    ancestors_name = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
    ancestors_label = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

else:
    print(f"Your {selected_dataset} doesn't exist. Please provide an existing dataset between these choices : CIFAR10, CIFAR100, ImageNet & MNIST.")

---------------------------------------------------------------------
**Fix seed to ensure reproducibility (comment to get random results)**

In [ ]:
np.random.seed(12345)

-----------------------

#### plot_prediction()

This function will be used to display the original / attacked images.

The function is designed to display the images correctly, depending on the dataset selected, with the following legend:

<font color='green'>Green bars</font> = correct classification <br>
<font color='red'>Red bars</font> = Attack target classification <br>
<font color='blue'>Blue bars</font> = other classifications

In [ ]:
def label_to_name_dynamic(index, dataset):
    """Retourne le nom du label en fonction du dataset sélectionné."""
    if dataset == "MNIST":
        return str(index)  # For MNIST, the label name is simply the digit
    elif dataset == "ImageNet":
        return label_to_name(index)  # Use the imagenet_stubs function for ImageNet !
    elif dataset == "CIFAR10":
        return ancestors_name[index]  # Return the name from our list
    elif dataset == "CIFAR100":
        return cifar100_labels[index] if 0 <= index < 100 else "Unknown" # Return the name from our list 
    else:
        return "Unknown"

In [ ]:
def plot_prediction(img, probs, correct_class=None, target_class=None):
    """
    Displays an image with predictions in the form of coloured bars :
    - Green --> Correct Class
    - Red --> Target Class
    - Blue --> Other Classes
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 8))

    # Display the picture
    if selected_dataset=="MNIST":
        ax1.imshow(img, cmap="gray") # Force the display in gray level for MNIST !
        ax1.axis("off")
    else:
        ax1.imshow(img)
        ax1.axis("off")

    # Keep the top 10 classes with highest probabilities
    top_ten_indexes = list(probs[0].argsort()[-10:][::-1])
    top_probs = probs[0, top_ten_indexes]
    labels = [label_to_name_dynamic(i, selected_dataset) for i in top_ten_indexes]


    # Bar plot creation with color rules defined above
    barlist = ax2.bar(range(10), top_probs, color="blue")  # Blue by default

    if target_class in top_ten_indexes:
        barlist[top_ten_indexes.index(target_class)].set_color("red")  # Red if this is the target class

    if correct_class in top_ten_indexes:
        barlist[top_ten_indexes.index(correct_class)].set_color("green")  # Green if this is the correct class

    # Plot Graph
    plt.sca(ax2)
    plt.ylim([0, 1.1])
    plt.xticks(range(10), labels, rotation="vertical")
    plt.ylabel("Probability")
    plt.title("Top 10 Predictions")
    fig.subplots_adjust(bottom=0.2)

    plt.show()

## Step 1: Define Parameters

### **Adversarial Patch Paper Parameters**

In the original **Adversarial Patch paper** (Brown et al., 2017), the authors proposed a **physically realizable attack** by training a universal patch on natural images with the following typical parameters:

- **Scale** → Range of patch size relative to the image:
  - `scale_min = 0.4` (smallest size = 40% of image)
  - `scale_max = 1.0` (up to full image size)

- **Rotation** → Max random rotation applied during training:
  - `rotation_max = 22.5` degrees

- **Learning Rate** → Patch optimization speed:
  - `learning_rate = 5000.0`

- **Iterations** → Number of gradient descent steps:
  - `max_iter = 250`

- **Batch Size** → Number of images used per optimization step:
  - `batch_size = 16`

> 🧠 These parameters control how flexible and robust the patch becomes.
Feel free to **tune them depending on your dataset or desired strength of the patch**.

In [ ]:
# Adversarial Patch Parameters (from original paper)

scale_min = 0.4          # Minimum scale of patch during training (40% of image size)
scale_max = 1.0          # Maximum scale (up to full image size)
rotation_max = 22.5      # Maximum rotation in degrees
learning_rate = 5000.0   # Learning rate for patch optimization
max_iter = 25           # Number of optimization iterations
batch_size = 16          # Number of images per optimization step

print(f"Selected Attack: Adversarial Patch | Dataset: {selected_dataset} | Scale Range: [{scale_min}, {scale_max}] | Rotation Max: {rotation_max}° | Batch Size: {batch_size}, Learning Rate: {learning_rate} | Max Iterations: {max_iter}")

-----------------------------
**Later, we'll see what EoT is. If you don't know what is EoT, skip this sub-section**

*If you want to test EoT Transformation, find parameters below*

In [ ]:
# Parameters for EoT Transformation
angle_max = 22.5 # Rotation angle used for evaluation in degrees
eot_angle = angle_max # Maximum angle for sampling range in EoT rotation, applying range [-eot_angle, eot_angle]
eot_samples = 10 # Number of samples with random rotations in parallel per loss gradient calculation

### Dataset-Specific Parameters

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

# ImageNet has 1000 classes, CIFAR100 100 classes, and CIFAR10 & MNIST has 10 classes.
nb_classes = 1000 if selected_dataset == "ImageNet" else 100 if selected_dataset == "CIFAR100" else 10

# ImageNet Images Dimension : (299,299,3), CIFAR10 & CIFAR100 : (32,32,3), and MNIST : (28,28,1)
input_shape = (224, 224, 3) if selected_dataset == "ImageNet" else (32, 32, 3) if "CIFAR" in selected_dataset else (28, 28, 1)
print(input_shape)
# ImageNet use often a specific preprocessing. For the others dataset, it still an adapted normalisation (0,1)
preprocessing = None if selected_dataset == "ImageNet" else (0.0, 1.0)  # Normalisation adaptée

# Clip values 
clip_values = (0.0, 1.0)  # Same for all datasets

# Target Class Definition (You can change, here are just some examples)
if selected_dataset == "ImageNet":
    y_target = np.array([641])  # "maraca"
elif selected_dataset == "CIFAR100":
    y_target = np.array([3])  # "bear"
elif selected_dataset == "CIFAR10":
    y_target = np.array([1])  # "automobile"
else:  # MNIST
    y_target = np.array([np.random.randint(0, 10)])  # random digit between 0 and 9

## Step 2: Load Dataset Data & Labels

In this step, we **load all dataset images and their labels into memory**.

#### **How does it work?**
1. We retrive the dataset path (`datasets/selected_dataset/`).
2. We read all images from the dataset folders.
3. We **normalize** the images (scale pixel values between `[0, 1]`).
4. We store **both images and labels** for further processing.

 -------------------

In [ ]:
# List Initializations
x_all, y_all, original_images = [], [], []

In [ ]:
# Try to get our dataset path in our computer to keep all pictures and put them into our lists.
dataset_path = os.path.join("datasets", selected_dataset)
# Check
assert(dataset_path=="datasets/"+selected_dataset) # If nothing : It's ok. Otherwise, you will get an error if the dataset path doesn't exists.

In [ ]:
# Load images from the selected dataset
for class_name, class_label in zip(ancestors_name, ancestors_label):
    class_path = os.path.join(dataset_path, class_name)
    if not os.path.exists(class_path):
        continue
    
    for img_file in sorted(os.listdir(class_path)):
        img_path = os.path.join(class_path, img_file)

        if selected_dataset == "MNIST":
            im = load_img(img_path, color_mode="grayscale", target_size=(28, 28))
            im_array = img_to_array(im)
        
        elif selected_dataset == "ImageNet":
            im = load_img(img_path, target_size=(224, 224))
            im_array = img_to_array(im)

        elif selected_dataset in ["CIFAR10", "CIFAR100"]:
            im = load_img(img_path, target_size=(32, 32))
            im_array = img_to_array(im)
        
        x = (im_array / 255.0).astype(np.float32)
        
        x_all.append(x)
        y_all.append(int(class_label))
        original_images.append(im_array)

-----------------------------------------------------
#### Display Dataset (Optional)
**You can choose to display all images or only one image per class)**

#### How to enable visualization ?
- To display **ALL images** --> **Uncomment the loop bellow**.
- To display **ONLY 1 image per class** --> **Set `display_all_images = False`**.
-----------------------------------------------------

```Python
# Set to True to display all images, False to show only 1 image per class
display_all_images = False  

# Displaying of the 100 pictures (can be long, you can modify the code to display only 1 picture per class if you want)
for class_name, class_label in zip(ancestors_name, ancestors_label):
    class_path = os.path.join(dataset_path, class_name)
    if not os.path.exists(class_path):
        print(class_path)
        print("No os Path")
        continue
    
    print(f"Class : {class_name} (Label: {class_label})")
    
     # Show only 1 image per class if display_all_images = False
    images_to_show = sorted(os.listdir(class_path))[:1] if not display_all_images else sorted(os.listdir(class_path))
    # Go through the 10 pictures of each classes
    for img_file in images_to_show:
        img_path = os.path.join(class_path, img_file)

        # Load & Normalize the picture
        im = load_img(img_path, target_size=(299, 299))
        im_array = img_to_array(im)

        # Displaying all pictures
        plt.figure(figsize=(4, 4))
        plt.imshow(im_array.astype("uint8"))
        plt.axis("off")
        plt.title(f"Class: {class_name} | {img_file}", fontsize=10, fontweight="bold")
        plt.show()

        print(f"{img_file} well displayed in : {class_name}")

print(f"All of the {len(ancestors_name)} classes & their images has been displayed !")
```

### Convert to Numpy Arrays for TensorFlow
Since TensorFlow requires NumPy arrays, we convert our lists into arrays.

-----------------

In [ ]:
# Convert into a numpy array
x_all = np.array(x_all)
y_all = np.array(y_all).astype(int).flatten()

# Check
#for img_x, img_y in zip(x_all, y_all):
#    print(f"x_all shape: {x_all.shape}")  # (N, H, W, C)
#    print(f"y_all shape: {y_all.shape}")  # (N, 1)

## Step 3 : Load Model & Loss Function

### 1. Loading Dataset for Model Training
Before creating the model, we **load and preprocess** the dataset to ensure it is correctly formatted for TensorFlow.

-------------

In [ ]:
if selected_dataset == "MNIST":
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = np.expand_dims(x_train, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)

elif selected_dataset == "CIFAR10":
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = to_categorical(y_train, nb_classes), to_categorical(y_test, nb_classes)
    
elif selected_dataset == "CIFAR100":
    (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode="fine")

    # We reproduce the list of all classes of CIFAR100
    cifar100_labels = [
    "apple", "aquarium_fish", "baby", "bear", "beaver", "bed", "bee", "beetle", "bicycle", "bottle",
    "bowl", "boy", "bridge", "bus", "butterfly", "camel", "can", "castle", "caterpillar", "cattle",
    "chair", "chimpanzee", "clock", "cloud", "cockroach", "couch", "crab", "crocodile", "cup", "dinosaur",
    "dolphin", "elephant", "flatfish", "forest", "fox", "girl", "hamster", "house", "kangaroo", "computer_keyboard",
    "lamp", "lawn_mower", "leopard", "lion", "lizard", "lobster", "man", "maple_tree", "motorcycle", "mountain",
    "mouse", "mushroom", "oak_tree", "orange", "orchid", "otter", "palm_tree", "pear", "pickup_truck", "pine_tree",
    "plain", "plate", "poppy", "porcupine", "possum", "rabbit", "raccoon", "ray", "road", "rocket", "rose", "sea",
    "seal", "shark", "shrew", "skunk", "skyscraper", "snail", "snake", "spider", "squirrel", "streetcar", "sunflower",
    "sweet_pepper", "table", "tank", "telephone", "television", "tiger", "tractor", "train", "trout", "tulip",
    "turtle", "wardrobe", "whale", "willow_tree", "wolf", "woman", "worm"
]

    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = to_categorical(y_train, nb_classes), to_categorical(y_test, nb_classes)

### 2. Model Selection & Architecture
Different models are used depending on the selected dataset:
- **ImageNet** --> **InceptionV3**

In [ ]:
# ============= IMAGENET =============
if selected_dataset == "ImageNet":
    print(f"SELECTED MODEL : ResNet50")  
    model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

# ============= ERROR =============
else:
    raise ValueError(f"Error: Dataset '{selected_dataset} not recognized. Please ensure to use one of this dataset : ImageNet, CIFAR10, CIFAR100 or MNIST.'")

## Step 4 : Create the ART Classifier & Configure the Attack

Now that the model is **trained and ready**, we integrate it into **ART (Adversarial Robustness Toolbox)**.

#### What is happening here ?
1. We **create a classifier** for ART based on the trained model
2. We **define an adversarial attack** (FGSM in this case)
3. The attack can be **targeted or untargeted**, and parameters are fully configurable.

In [ ]:
classifier = TensorFlowV2Classifier(model=model,
                                    nb_classes=1000,
                                    input_shape=(224, 224, 3),
                                    clip_values=(0, 255),
                                    preprocessing = ([103.939, 116.779, 123.68], [1.0, 1.0, 1.0]),  # BGR mean values
                                    loss_object=tf.keras.losses.CategoricalCrossentropy(from_logits=False)
                                   )

In [ ]:
attack = AdversarialPatch(
    classifier=classifier,
    rotation_max=rotation_max,
    scale_min=scale_min,
    scale_max=scale_max,
    learning_rate=learning_rate,
    max_iter=max_iter,
    batch_size=batch_size,
    verbose=True
)

---

**ONE-HOT ENCODING (Y_TARGET)**

---

In [ ]:
y_one_hot = np.zeros(1000)
y_one_hot[y_target] = 1.0

---

**REPEAT TARGET FOR ALL IMAGES**

---

In [ ]:
# Then we repeat the same target for all images.
y_target_one_hot_encoded = np.tile(y_one_hot, (x_all.shape[0], 1))

## Step 5 : Predict Clean (Original) Images BEFORE the attack.

Before applying any attack, we **predict the clean images** with our trained model.

#### What happens here ?
1. We run the classifier on all images **before the attack**.
2. We display the **top-10 predictions** for each image.
3. You can choose to **display all images or only one per class**.

------------

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

# Apply specific process for ResNet50
x_all_preprocessed = preprocess_input(x_all.copy() * 255.0) 

# Predict using the classifier
y_pred_clean_all = classifier.predict(x_all_preprocessed)

# Check prediction shape
print("Shape of Clean Predictions:", y_pred_clean_all.shape)  # Expected (N, nb_classes)

top1_correct = np.mean(np.argmax(y_pred_clean_all, axis=1) == y_all) * 100
print(f"Top-1 Accuracy on Clean Images: {top1_correct:.2f}%")

---------------------------

#### Display Clean Images & Predictions
You can **choose whether to display all images or just one per class**.

**How enable visualization?**
- To display **ALL images** --> Set `display_all_images = True`
- To display **ONLY 1 image per class** --> Set `display_all_images = False`

---------------------

In [ ]:
# Set to True to display all images, False to show only 1 image per class
display_all_images = False  

# Displaying Clean Images with Predictions
for class_name, class_label in zip(ancestors_name, ancestors_label):
    print(f"\nClass : {class_name} (Label: {class_label})")

    # Get all images from this class
    class_indices = np.where(y_all == int(class_label))[0]

    if len(class_indices) == 0:
        print(f"No Images found for {class_name}, skipping...")
        continue

    # Show only 1 image per class if display_all_images = False
    images_to_show = class_indices[:1] if not display_all_images else class_indices
    
    for index in images_to_show:
        plot_prediction(
            np.squeeze(x_all[index]),  # Original clean image
            y_pred_clean_all[index].reshape(1, -1),  # Reshaped prediction
            correct_class=y_all[index],  # True class
            target_class=None  # No target class for clean images
        )
        print(f"Image {index} displayed for class: {class_name}")

print(f"\n All {len(x_all)} clean images have been processed!")

## Step 6: Generate and Evaluate Adversarial Examples

Now, we **generate adversarial examples** and evaluate the effectiveness of the attack.

 **What happens here?**
1. We **generate adversarial examples** using the selected attack.
2. We **save the adversarial images** for later analysis. (optional)
3. We **evaluate the attack's success** (accuracy, confidence scores, and performance metrics).
4. We **generate a detailed report** summarizing the attack results.

---

--- 

### GENERATE THE PATCH

---

In [ ]:
patch, patch_mask = attack.generate(x=x_all, y=y_target_one_hot_encoded)

--- 

### GENERATE THE ATTACK

---

In [ ]:
patched_images = attack.apply_patch(x_all, scale=0.5)

# Prediction on Adversarial (Patched) Images
y_pred_adv_all = classifier.predict(patched_images)

---------------------

**Do you want to save all adversarial images?**  
- **YES** → Uncomment the saving function below.
- **NO** → Comment the function to skip saving.


```Python
# Define the save path for adversarial images
adv_save_path = os.path.join("adversarials_img", selected_attack, selected_dataset)
os.makedirs(adv_save_path, exist_ok=True)  

# Iterate through all classes to save adversarial images
class_counters = {class_name: 1 for class_name in ancestors_name}  # Dictionary to track image indices per class

for adv_img, class_label in zip(x_adv_all, y_all.flatten()):  # Ensure y_all is 1D
    # Find the class name corresponding to the label
    if str(class_label) not in ancestors_label:
        print(f"Label {class_label} not found in ancestors_label, skipping image.")
        continue  

    class_index = ancestors_label.index(str(class_label))
    class_name = ancestors_name[class_index]

    # Determine the subfolder for the class
    class_folder = os.path.join(adv_save_path, class_name)
    os.makedirs(class_folder, exist_ok=True) 

    # Generate a unique filename with a counter (e.g., abacus1_adv.jpeg, abacus2_adv.jpeg, ..., acorn1_adv.jpeg, ...)
    img_filename = f"{class_name}{class_counters[class_name]:02d}_adv.jpeg"
    img_path = os.path.join(class_folder, img_filename)

    # Convert and save the image
    img = array_to_img(adv_img)
    img.save(img_path, "JPEG")

    print(f"Image saved : {img_path}")

    # Increment the counter for this class
    class_counters[class_name] += 1

print(f"\nAll  {len(x_adv_all)} adversarial images have been successfully saved!")
```

---

### Evaluate Adversarial Example

We now evaluate the adversarial examples by:
- Measuring the **model's accuracy** on these images.
- Computing the **confidence score** of predictions.
- Generating a **visual comparison** between clean and adversarial images.

---

**How enable visualization?**
- To display **ALL images** --> Set `display_all_images = True`
- To display **ONLY 1 image per class** --> Set `display_all_images = False`

---

In [ ]:
# Set to True to display all images, False to show only 1 image per class
display_all_images = False

for class_name, class_label in zip(ancestors_name, ancestors_label):
    print(f"\nClass : {class_name} (Label: {class_label})")

    class_indices = np.where(y_all == int(class_label))[0]
    if len(class_indices) == 0:
        print(f"No images found for {class_name}, skipping...")
        continue

    images_to_show = class_indices[:1] if not display_all_images else class_indices

    for index in images_to_show:
        plot_prediction(
            np.squeeze(patched_images[index]),
            y_pred_adv_all[index].reshape(1, -1),
            correct_class=y_all[index],
            target_class=y_target
        )
        print(f"Adversarial Image {index} displayed for class: {class_name}")

### Compute Performance Metrics

In [ ]:
# Compute confidence score
confidence_scores = np.max(y_pred_clean_all, axis=1)
average_confidence = np.mean(confidence_scores) * 100

# Compute Tok-K Accuracy
def compute_accuracy(predictions, true_labels, top_k=1):
    top_k_preds = np.argsort(predictions, axis=1)[:, -top_k:]
    match = np.any(top_k_preds == np.array(true_labels).reshape(-1, 1), axis=1)
    return np.mean(match) * 100 

In [ ]:
clean_top1 = compute_accuracy(y_pred_clean_all, y_all, top_k=1)
clean_top5 = compute_accuracy(y_pred_clean_all, y_all, top_k=5)
adv_top1 = compute_accuracy(y_pred_adv_all, y_all, top_k=1)
adv_top5 = compute_accuracy(y_pred_adv_all, y_all, top_k=5)

In [ ]:
# Display Performance Results
attack_name = "AdversarialPatch" if isinstance(attack, AdversarialPatch) else "iter. basic"

In [ ]:
print("\n=== Performance Summary ===")
print(f"Selected Attack: Adversarial Patch | Dataset: {selected_dataset} | Scale Range: [{scale_min}, {scale_max}] | Rotation Max: {rotation_max}° | Batch Size: {batch_size}, Learning Rate: {learning_rate} | Max Iterations: {max_iter}")
print(f"Clean Images : Top-1 : {clean_top1:.2f}% | Top-5 : {clean_top5:.1f}%")
print(f"Adv. Images  : Top-1 : {adv_top1:.2f}% | Top-5 : {adv_top5:.1f}%")
print("----------------------------------------------------------")
print(f"Confidence Score: {average_confidence:.2f}%")

### Generate a Report

In [ ]:
# Round epsilon for eadability
#eps_rounded = round(epsilon, 3)

# Define report save path
if selected_dataset == "MNIST":
    report_filename = f"{selected_attack}_with_{selected_dataset}_with_{selected_mnist_model}_report.txt"
elif selected_dataset in ["CIFAR10", "CIFAR100"]:
    report_filename = f"{selected_attack}_with_{selected_dataset}.txt"
elif selected_dataset == "ImageNet":
    report_filename = f"{selected_attack}_with_{selected_dataset}_with_ResNet50_report.txt"
else:
    print(f"This {selected_dataset} is not recognized. Be careful to provide an existing dataset between MNIST, CIFAR")
    
report_path = os.path.join("adversarials_img", selected_attack, selected_dataset, report_filename)
os.makedirs(os.path.dirname(report_path), exist_ok=True)


with open(report_path, "w", encoding="utf-8") as f:
    # Report Title
    f.write(f"====== {selected_attack} Adversarial Attack Report) ======\n\n")

    # Information generation for each image
    for i in range(len(y_pred_adv_all)):  
        # Find the class index in ancestors_label
        class_label = str(int(y_all[i]))
        if class_label in ancestors_label:
            class_index = ancestors_label.index(class_label)  # Get index in ancestors_name
            class_name = ancestors_name[class_index]  # Retrieve class name
        else:
            class_name = "Unknown"  # If not found, prevent error

        # Original Image file name (ensuring correct numbering)
        original_image_name = f"{class_name}{(i % 10) + 1:02d}.jpeg"

        # Predict Class for the original image (top-1)
        clean_pred_index = np.argmax(y_pred_clean_all[i])

        # Predict Class for the Adversarial image (top-1)
        adv_pred_index = np.argmax(y_pred_adv_all[i])

        # Prediction
        clean_pred_label = label_to_name_dynamic(clean_pred_index, selected_dataset)
        adv_pred_label = label_to_name_dynamic(adv_pred_index, selected_dataset)


        # Targeted or Untargeted Scenario Attack
        attack_type = "Targeted" if attack.targeted else "Untargeted"

        # If Targeted : Target Class
        target_label = label_to_name(y_target[0]) if attack.targeted else "N/A"

        # Write results in the report:
        f.write(f"------ CLASS : {class_name.upper()} ------\n")
        f.write(f"Original image name : {original_image_name}\n")
        f.write(f"Original Prediction : {clean_pred_label}\n")
        f.write(f"Targeted / Untargeted : {attack_type}\n")
        if attack.targeted:
            f.write(f"Target Class : {target_label}\n")
        f.write(f"Adversarial Prediction : {adv_pred_label}\n")
        f.write("------------------------------------------------\n\n")

    # Performance Summary at the end of the file
    f.write("============ PERFORMANCE RESUME ============\n")
    f.write(f"Selected Attack: Adversarial Patch | Dataset: {selected_dataset} | Scale Range: [{scale_min}, {scale_max}] | Rotation Max: {rotation_max}° | Batch Size: {batch_size}, Learning Rate: {learning_rate} | Max Iterations: {max_iter}")
    f.write(f"Clean Images : Top-1 : {clean_top1:.1f}% | Top-5 : {clean_top5:.1f}%\n")
    f.write(f"Adv. Images  : Top-1 : {adv_top1:.1f}% | Top-5 : {adv_top5:.1f}%\n")
    f.write("----------------------------------------------------------")
    f.write(f"Confidence Score: {average_confidence:.2f}%")

    attack_eff_top1 = 100 - adv_top1
    attack_eff_top5 = 100 - adv_top5

    f.write("\n")
    f.write(f"{selected_attack} Efficiency : Top-1 : {attack_eff_top1:.1f}% | Top-5 : {attack_eff_top5:.1f}%\n")

# Saving Confirmation
print(f"Report saved : {report_path}")

# Going further (optional) : Expectation Over Transformation (EoT) 
Adversarial attacks like **FGSM** are often **sensitive to image transformations** such as **rotation, scaling, or noise**.

**Why does this happen?**  
- A small rotation (e.g., **5°**) can **invalidate** an adversarial example.
- This **breaks the perturbation pattern** that misleads the classifier.
  
**How does EoT (Expectation Over Transformation) help?**  
- Instead of using **a single perturbed image**, EoT **randomly transforms** the image (rotation, blur, etc.).
- The attack is then **optimized over multiple transformations**, making it **more robust**.

---

In [ ]:
import scipy.ndimage

# Define rotation angles to test
rotation_angles = [-22.5, -10.0, -5.0, 0.0, 5.0, 10.0, 22.5]  

# Apply rotation to all adversarial examples
x_adv_rotated_all = {
    angle: np.array([
        scipy.ndimage.rotate(img, angle=angle, reshape=False, axes=(0, 1), order=1, mode='constant')
        for img in x_adv_all
    ]) for angle in rotation_angles
}

# Get predictions after rotation
y_pred_adv_rotated_all = {
    angle: classifier.predict(x_adv_rotated_all[angle])
    for angle in rotation_angles
}

print(f"Adversarial images rotated and evaluated for {len(rotation_angles)} angles.")


### Display Rotated Adversarial Examples
You can **choose whether to display all images or just a few.**

In [ ]:
display_all_images = False  # Set to True to display all, False to show a few per angle

for angle in rotation_angles:
    print(f"\nRotation Angle: {angle}°")

    for i in range(len(x_adv_rotated_all[angle])):
        if not display_all_images and i > 1:
            break

        plot_prediction(
            np.squeeze(x_adv_rotated_all[angle][i]),  
            y_pred_adv_rotated_all[angle][i].reshape(1, -1),  
            correct_class=y_all[i],  
            target_class=y_target  
        )

### Evaluate Performance After Rotation

In [ ]:
# Compute Accuracy After Rotation
for angle in rotation_angles:
    adv_top1_rotated = compute_accuracy(y_pred_adv_rotated_all[angle], y_all, top_k=1)
    adv_top5_rotated = compute_accuracy(y_pred_adv_rotated_all[angle], y_all, top_k=5)
    
    print(f"Rotation {angle}° → Top-1: {adv_top1_rotated:.1f}% | Top-5: {adv_top5_rotated:.1f}%")

## Step 7: Apply Expectation Over Transformation (EoT)

### **What is EoT and Why is it Useful?**
FGSM and adversarial attacks often **fail** when images undergo transformations like **rotations**.

**EoT (Expectation Over Transformation) mitigates this issue by:**
- Generating multiple **randomly transformed** versions of the adversarial image.
- Applying these transformations **during model evaluation** (predictions & gradients).
- Making the adversarial attack **robust to transformations** like **rotations, noise, and blur**.

---

### **Enable EoT in ART**
We use ART’s **`EoTImageRotationTensorFlow`** to introduce **random rotations** during classification.

---


In [ ]:
# Create ART Classifier with EoT
eot_rotation = EoTImageRotationTensorFlow(nb_samples=eot_samples,  
                                          clip_values=clip_values,  
                                          angles=eot_angle)  # Random rotation range

classifier_eot = TensorFlowV2Classifier(model=model,
                                        nb_classes=nb_classes,
                                        loss_object=tf.keras.losses.CategoricalCrossentropy(),
                                        preprocessing=preprocessing,
                                        preprocessing_defences=[eot_rotation],  # EoT applied
                                        clip_values=clip_values,
                                        input_shape=input_shape)

print(f"EoT Classifier created with {eot_samples} transformation samples per evaluation.")


### Generate Adversarial Examples with EoT
We generate **adversarial examples** that remain effective even **after transformations**.

-----------------

In [ ]:
from tqdm import tqdm

# Prepare target labels for targeted attacks
y_target_one_hot = np.zeros((1, nb_classes), dtype=np.float32)
y_target_one_hot[0, name_to_label("guacamole")] = 1.0  
y_target_all = np.tile(y_target_one_hot, (len(x_all), 1))  

x_adv_eot_all = []

for i in tqdm(range(len(x_all)), desc="Generating EoT Examples"):
    x_i = np.expand_dims(x_all[i], axis=0)  
    y_i = np.expand_dims(y_target_all[i], axis=0)  

    if attack.targeted:
        x_adv_i = attack.generate(x=x_i, y=y_i)
    else:
        x_adv_i = attack.generate(x=x_i)

    x_adv_eot_all.append(np.squeeze(x_adv_i))  

x_adv_eot_all = np.array(x_adv_eot_all)

print(f"Shape of EoT Adversarial Examples: {x_adv_eot_all.shape}")

### Apply Rotation to Adversarial Examples
We now test the **robustness** of these adversarial examples by **rotating them** at different angles.

--------------------

In [ ]:
# Define rotation angles
rotation_angles = [-22.5, -10.0, -5.0, 0.0, 5.0, 10.0, 22.5]  

# Rotate and Evaluate Adversarial Examples
x_adv_rotated_all = {
    angle: np.array([
        scipy.ndimage.rotate(img, angle=angle, reshape=False, axes=(1, 2), order=1, mode='constant')
        for img in x_adv_eot_all
    ]) for angle in rotation_angles
}

y_pred_adv_rotated_all = {
    angle: classifier.predict(x_adv_rotated_all[angle])
    for angle in rotation_angles
}

print(f"Adversarial images rotated and evaluated for {len(rotation_angles)} angles.")

### Display Rotated Adversarial Examples
You can **choose whether to display all images or just a few per rotation angle**.

----------

In [ ]:
display_all_images = False  

for angle in rotation_angles:
    print(f"\nRotation Angle: {angle}°")

    for i in range(len(x_adv_rotated_all[angle])):
        if not display_all_images and i > 1:  
            break

        plot_prediction(
            np.squeeze(x_adv_rotated_all[angle][i]),  
            y_pred_adv_rotated_all[angle][i].reshape(1, -1),  
            correct_class=y_all[i],  
            target_class=y_target  
        )

### Evaluate Performance After Rotation

In [ ]:
# Compute Accuracy After Rotation
for angle in rotation_angles:
    adv_top1_rotated = compute_accuracy(y_pred_adv_rotated_all[angle], y_all, top_k=1)
    adv_top5_rotated = compute_accuracy(y_pred_adv_rotated_all[angle], y_all, top_k=5)
    
    print(f"Rotation {angle}° → Top-1: {adv_top1_rotated:.1f}% | Top-5: {adv_top5_rotated:.1f}%")

### Generate a Report on EoT Performance

In [ ]:
# Define report save path
report_filename = f"EoT_{selected_attack}_{selected_dataset}_eps={round(epsilon, 2)}.txt"
report_path = os.path.join("adversarials_img", selected_attack, selected_dataset, report_filename)
os.makedirs(os.path.dirname(report_path), exist_ok=True)

# Generate Report
print("\nGenerating EoT attack report...")

with open(report_path, "w", encoding="utf-8") as f:
    f.write(f"====== EoT Adversarial Attack Report (ε = {round(epsilon, 2)}) ======\n\n")
    
    for angle in rotation_angles:
        adv_top1_rotated = compute_accuracy(y_pred_adv_rotated_all[angle], y_all, top_k=1)
        adv_top5_rotated = compute_accuracy(y_pred_adv_rotated_all[angle], y_all, top_k=5)
        
        f.write(f"\n=== Rotation {angle}° ===\n")
        f.write(f"Top-1 Accuracy: {adv_top1_rotated:.1f}%\n")
        f.write(f"Top-5 Accuracy: {adv_top5_rotated:.1f}%\n")

print(f"EoT Report saved: {report_path}")